# Segmenting and Clustering Neighbourhoods in Toronto

# Part 1

## 1.0 Obtaining neighbourhoods data from Wikipedia 

In [1]:
#Library Import
import pandas as pd # library for data analsysis 
import numpy as np

In [2]:
#This packagae nee to be installed in order to read the html
!pip install lxml
!pip install html5lib
!pip3 install lxml
import lxml


In [3]:
#Scraping wikipedia table to pandas data frame
import requests
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Data Cleaning

### 1.1 Ignore cells with a borough that is Not assigned

In [4]:
#Ignore cells with a borough that is Not assigned
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df=df.reset_index(drop= True) #resetting the index
df.head(10)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### 1.2 Merge Boroughs that have the same postal code in a same row.

In [5]:
#Merge Boroughs that have the same postal code in a same row. Separate by comma
df_borough=df.groupby('Postal Code').agg({'Borough':'first','Neighbourhood':', '.join}).reset_index()
df_borough.sort_values(by=['Postal Code']).head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### 1.3 Assign the name of the Borough to not assiged neighbourhood

In [6]:
#Assign the name of the Borough to not assiged neighbourhood

#Find the Not assigned Neighbourhood
NA_Neighborhood= df_borough['Neighbourhood'] == 'Not assigned'
#NA_Neighborhood[NA_Neighborhood ==False].shape[0] #Count where this condition apply

#Assign Borough name to the neighbourhood that are Not Assigned
df_borough.loc[NA_Neighborhood, 'Neighbourhood'] = df_borough.loc[NA_Neighborhood, 'Borough'] 
df_borough.head()



,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### 1.4 Print Row of the DataFrame

In [7]:
df_borough.shape

(103, 3)

# Part 2

## 2.0 Get the latitude and the longitude coordinates of each neighborhood

In [8]:
#Import the library
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

# All requested packages already installed.



In [9]:
#Install and import geoconder
!pip install geocoder
import geocoder # import geocoder


In [10]:
#Install and import BeautifulSoup
!pip install bs4
from bs4 import BeautifulSoup

In [14]:
#Getting the coordinates 
df_coordinate= pd.read_csv('https://cocl.us/Geospatial_data')
df_coordinate.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
#Merging the dataframes
df_merged=pd.merge(df_borough, df_coordinate, on="Postal Code")
df_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
